In [ ]:
import os

from glob import glob

%load_ext autoreload
%autoreload 1

%aimport homstrad_util

# Split PDB chains into single PDB files

In [ ]:
for family in tqdm(os.listdir('homstrad_db/')):
    os.makedirs(f'out/pdbs/{family}', exist_ok=True)
    
    # Parse family member names
    with open(f'homstrad_db/{family}/{family}.ali') as file:
        fam_members = []
        for line in file:
            if line.startswith('>'):
                name = line[4:-1]
                fam_members.append(name)
                assert len(name) != 0, f'{fam_members[-1]} is to short'
        
    
    # Split family PDB
    chains = ''  # 
    with open(f'homstrad_db/{family}/{family}-sup.pdb') as file:
        single_pdbs = [[] for _ in fam_members]
        last_chain = None
        i = -1
        for line in file:
            if line.startswith('ATOM'):
                chain = line[21:22]
                if chain != last_chain:
                    i += 1
                    last_chain = chain
                    chains += chain
                assert i >= 0
                single_pdbs[i].append(line)

    assert len(chains) == len(single_pdbs)
                
    # Find mapping between alignment and pdb file
    with open(f'homstrad_db/{family}/{family}-sup.pdb') as file:
        pdb_txt = file.read()
    # mapping is contained in remark section of pdb file
    if 'REMARK The domains in this file are:' in pdb_txt:
        lines = pdb_txt.splitlines()
        idx = lines.index([line for line in lines if line.startswith('REMARK The domains in this file are:')][0])
        id2chain = {}
        for line in lines[idx+1:idx+1+len(fam_members)]:
            _, name, chain, chain_name = line.split()
            assert chain == 'chain'
            id2chain[name] = chain_name

        
    # Extract all pdbs
    if 'REMARK The domains in this file are:' in pdb_txt:
        pdbs_ordered = [single_pdbs[chains.index(id2chain[name])] for name in fam_members]
    else:
        pdbs_ordered = single_pdbs
    for lines, name in zip(pdbs_ordered, fam_members):
        assert len(lines) > 0
        os.makedirs(f'out/hmstrd_pdbs/{family}', exist_ok=True)
        with open(f'out/hmstrd_pdbs/{family}/{name}.pdb', 'w') as pdb_file:
            pdb_file.writelines(lines)

# Run DALI

In [ ]:
%%writefile run_dali.sh
#!/bin/bash

DALIBIN=$(realpath $1)
FAMDIR=$2

rm -rf "$FAMDIR"/dali_tmp
mkdir -p "$FAMDIR"/dali_tmp/DAT
mkdir -p "$FAMDIR"/dali_tmp/out

# Import pdbs into Dali0
for name in "$FAMDIR"/*.pdb
do    
    BN="${name##*/}";
    "$DALIBIN"/import.pl --pdbfile $name --pdbid "${BN:0:4}" --clean --dat "$FAMDIR"/dali_tmp/DAT >> "$FAMDIR"/dali_tmp/log.txt 2>&1
done

# Create list of imported pdbs
for name in "$FAMDIR"/dali_tmp/DAT/*.dat
do    
    BN="${name##*/}";
    echo "${BN:0:5}" >> "$FAMDIR"/dali_tmp/ids.txt
done

cd "$FAMDIR"/dali_tmp/out
"$DALIBIN"/dali.pl --query ../ids.txt --matrix --dat1 ../DAT --clean --outfmt "summary,alignments" >> ../log.txt 2>&1

In [ ]:
!chmod +x run_dali.sh

In [ ]:
for fam in os.listdir('out/hmstrd_pdbs/'):
    path = f'out/hmstrd_pdbs/{fam}'
    !./run_dali.sh out/DaliLite.v5/bin {path}
    print('>'*10, path)

## Parse Dali output

In [ ]:
def normalize_dali_aln(aln1, aln2):
    assert len(aln1) == len(aln2)
    res1, res2 = '', ''
    buffer1, buffer2 = '', ''
    for c1, c2 in zip(aln1, aln2):
        if c1.islower() and c2.islower():
            #res1 += '-' + c1
            #res2 += c2 + '-'
            buffer1 += c1
            buffer2 += c2
        else:
            if buffer1:
                res1 += buffer1 + ('-' * len(buffer1))
                res2 += ('-' * len(buffer2)) + buffer2
                buffer1, buffer2 = '', ''
            res1 += c1
            res2 += c2

    if buffer1:
        res1 += buffer1 + ('-' * len(buffer1))
        res2 += ('-' * len(buffer2)) + buffer2
        buffer1, buffer2 = '', ''
    return res1, res2

def read_chain_name(family, name):
    with open(f'out/hmstrd_pdbs/{family}/{name}.pdb') as file:
        return file.readline().split()[4][:1]

def recover_name(family, dali_id):
    """Dali takes only the first 4 letters from the HOMSTRDA pdb names *and* appends the chain in the output."""
    # Read the original names 
    ref_aln = homstrad_util.parse_homstrad_ali(f'homstrad_db/{family}/{family}.ali')
    ref_names = [i[0] for i in ref_aln]

    # Find canditates where the first letters match
    hits = [n for n in ref_names if n.startswith(dali_id[:4])]
    if len(hits) == 1:
        return hits[0]
    else:
        # Additionally lookup the chain names in the pdb files
        chain_names = [read_chain_name(family, hit) for hit in hits]
        try:
            idx = chain_names.index(dali_id[4])
        except ValueError:
            print(f'dali_id: {dali_id}, hits: {hits}')
            raise
        return hits[idx]

def parse_dali(root_dir, family, query):
    # Locate result file
    dali_out = f'{root_dir}/{family}/dali_tmp/out'
    cand = glob(f'{dali_out}/{query}*.txt')
    if not cand:
        chain = read_chain_name(family, query)
        cand = glob(f'{dali_out}/{query[:4] + chain}*.txt')
    assert len(cand) == 1, f'{query} <-> {cand}'

    with open(cand[0]) as dali_output:
        txt = dali_output.read()

    # Split by target
    alignments = txt.split('No ')[1:]

    output = []
    for aln in alignments:
        lines = aln.splitlines()
        header = lines[0].split()
        qname, tname = header[1].split('=')[1], header[2].split('=')[1]
        qaln, taln = '', ''
        for line in lines:
            if line.startswith('Query'):
                qaln += line.split()[1]
            if line.startswith('Sbjct'):
                taln += line.split()[1]

        qaln, taln = normalize_dali_aln(qaln, taln)
        qname = recover_name(family, qname)
        tname = recover_name(family, tname)

        output.append(f'{qname} {tname} 0 0 {qaln} {taln}')
        
    # Write result
    with open(f'{root_dir}/{family}/dali.aln', 'w') as file:
        file.write('\n'.join(output))

In [ ]:
for fam in os.listdir('out/hmstrd_pdbs/'):
    query = homstrad_util.parse_homstrad_ali(f'homstrad_db/{fam}/{fam}.ali')[0][0]
    try:
        parse_dali('out/hmstrd_pdbs/', fam, query)
    except AssertionError as e:
        print(fam, e)

# Run Foldseek

In [ ]:
%%writefile run_foldseek.sh
#!/bin/bash

FOLDSEEK="$1"
FAMDIR="$2"

rm -rf "$FAMDIR"/foldseek_tmp
mkdir -p "$FAMDIR"/foldseek_tmp

$FOLDSEEK easy-search "$FAMDIR" "$FAMDIR" "$FAMDIR"/foldseek_tmp/result.tsv out/tmp/ \
    --exhaustive-search -e 100000 -a --format-output "query,target,qstart,tstart,qaln,taln,evalue,bits" >> "$FAMDIR"/foldseek_tmp/log.txt 2>&1

awk '{split($1, name1, "."); split($2, name2, "."); print name1[1], name2[1], $3-1, $4-1, $5, $6}' "$FAMDIR"/foldseek_tmp/result.tsv > "$FAMDIR"/foldseek.aln

In [ ]:
!chmod +x run_foldseek.sh

In [ ]:
for fam in os.listdir('out/hmstrd_pdbs/'):
    path = f'out/hmstrd_pdbs/{fam}'
    !./run_foldseek.sh ../../../git/foldseek/build/src/foldseek {path}

# Generate 3Di sequences for each family (AA sequence are in alignment file)

In [ ]:
%%bash --out out

FOLDSEEK=../../../git/foldseek/build/src/foldseek

rm -f out/db*

for fam in out/hmstrd_pdbs/*
do
    $FOLDSEEK createdb $fam out/db
    $FOLDSEEK lndb out/db_h out/db_ss_h
    $FOLDSEEK convert2fasta out/db_ss $fam/ss.fasta
done